# Deconvolution with DSEA

The Dortmund Spectrum Estimation Algorithm (DSEA) reconstructs the target distribution from classifier predictions on the target quantity of individual examples. CherenkovDeconvolution.jl implements the improved version DSEA+, which is extended by adaptive step sizes and a fixed reweighting of examples.

For a quick start, we deconvolve the distribution of Iris plant types in the famous IRIS data set.

In [1]:
# load example data
using MLDataUtils
X, y_labels, _ = load_iris()

# discretize the target quantity (for numerical values, we'd use LinearDiscretizer)
using Discretizers: encode, CategoricalDiscretizer
y = encode(CategoricalDiscretizer(y_labels), y_labels) # vector of target value indices

# have a look at the content of y
unique(y) # its just indices

3-element Array{Int64,1}:
 1
 2
 3

In [2]:
# Split the data into training and observed data sets.
# 
# The matrices MLDataUtils expects are transposed, by default.
# Thus, we have to be explicit about obsdim = 1. Note that
# CherenkovDeconvolution.jl follows the convention of ScikitLearn.jl
# (and others), which is size(X_train) == (n_examples, n_features).
# 
# MLDataUtils unfortunately assumes size(X_train) == (n_features, n_examples),
# but obsdim = 1 fixes this assumption.
# 
(X_train, y_train), (X_data, y_data) = splitobs(shuffleobs((X', y), obsdim = 1), obsdim = 1);

In [3]:
#
# Now let's estimate the target distribution!
#
using ScikitLearn, CherenkovDeconvolution

# deconvolve with a Naive Bayes classifier
@sk_import naive_bayes : GaussianNB
tp_function = Sklearn.train_and_predict_proba(GaussianNB())

f_est = dsea(X_data, X_train, y_train, tp_function)

INFO: Utilities of ScikitLearn.jl are available in CherenkovDeconvolution.Sklearn


3-element Array{Float64,1}:
 0.377778
 0.316244
 0.305978

In [4]:
# compare the result to the true target distribution, which we are estimating
f_true = Util.fit_pdf(y_data) # f_est is almost equal to f_true!

3-element Array{Float64,1}:
 0.377778
 0.333333
 0.288889